# Experiment 004: Small N Exhaustive Optimization

Goal: Focus on N=1,2,3 where we can do near-exhaustive search to find optimal configurations.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
from scipy.optimize import minimize, differential_evolution
import warnings
warnings.filterwarnings('ignore')

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_VERTICES = list(zip(TX, TY))

def create_tree_polygon(x, y, deg):
    poly = Polygon(TREE_VERTICES)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def get_bounding_box_side(polygons):
    if not polygons:
        return 0
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    return max(max(xs) - min(xs), max(ys) - min(ys))

def has_overlap(polygons):
    if len(polygons) < 2:
        return False
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-10:
                        return True
    return False

print("Setup complete")

Setup complete


In [2]:
# Load baseline solution
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv'
baseline_df = pd.read_csv(baseline_path)

def parse_s_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

baseline_df['x_val'] = baseline_df['x'].apply(parse_s_value)
baseline_df['y_val'] = baseline_df['y'].apply(parse_s_value)
baseline_df['deg_val'] = baseline_df['deg'].apply(parse_s_value)

print(f"Loaded baseline with {len(baseline_df)} rows")

# Analyze current small N configurations
print("\nCurrent small N configurations:")
for n in range(1, 11):
    prefix = f'{n:03d}_'
    group = baseline_df[baseline_df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    side = get_bounding_box_side(polygons)
    score = side**2 / n
    print(f"N={n:2d}: side={side:.6f}, score={score:.6f}")

Loaded baseline with 20100 rows

Current small N configurations:
N= 1: side=0.813173, score=0.661250
N= 2: side=0.949504, score=0.450779
N= 3: side=1.142031, score=0.434745
N= 4: side=1.290806, score=0.416545
N= 5: side=1.443692, score=0.416850
N= 6: side=1.548438, score=0.399610
N= 7: side=1.673104, score=0.399897
N= 8: side=1.755921, score=0.385407
N= 9: side=1.867280, score=0.387415
N=10: side=1.940696, score=0.376630


In [3]:
# N=1 optimization: Find the optimal rotation angle for a single tree
# The tree has a specific shape, so the bounding box depends on rotation

def get_single_tree_bbox_side(angle):
    """Get bounding box side for a single tree at given angle"""
    poly = create_tree_polygon(0, 0, angle)
    coords = list(poly.exterior.coords)
    xs = [c[0] for c in coords]
    ys = [c[1] for c in coords]
    return max(max(xs) - min(xs), max(ys) - min(ys))

# Search all angles from 0 to 360 in small increments
print("Searching for optimal N=1 rotation angle...")
best_angle = 0
best_side = float('inf')

for angle in np.linspace(0, 360, 3601):  # 0.1 degree increments
    side = get_single_tree_bbox_side(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"Best angle: {best_angle:.2f} degrees")
print(f"Best side: {best_side:.6f}")
print(f"Best score: {best_side**2:.6f}")

# Current N=1 configuration
current_n1 = baseline_df[baseline_df['id'] == '001_0'].iloc[0]
current_side = get_single_tree_bbox_side(current_n1['deg_val'])
print(f"\nCurrent angle: {current_n1['deg_val']:.2f} degrees")
print(f"Current side: {current_side:.6f}")
print(f"Current score: {current_side**2:.6f}")
print(f"Potential improvement: {current_side**2 - best_side**2:.6f}")

Searching for optimal N=1 rotation angle...


Best angle: 45.00 degrees
Best side: 0.813173
Best score: 0.661250

Current angle: 45.00 degrees
Current side: 0.813173
Current score: 0.661250
Potential improvement: 0.000000


In [ ]:
# N=2 optimization: Find optimal positions and angles for 2 trees
# This is a 6-dimensional optimization problem (x1, y1, a1, x2, y2, a2)
# But we can fix one tree at origin and optimize the other

def get_n2_score(params):
    """Calculate score for N=2 configuration"""
    x2, y2, a1, a2 = params
    poly1 = create_tree_polygon(0, 0, a1)
    poly2 = create_tree_polygon(x2, y2, a2)
    
    # Check for overlap
    if poly1.intersects(poly2) and not poly1.touches(poly2):
        intersection = poly1.intersection(poly2)
        if intersection.area > 1e-10:
            return 1000  # Penalty for overlap
    
    side = get_bounding_box_side([poly1, poly2])
    return side**2 / 2

# Grid search for N=2
print("Grid search for optimal N=2 configuration...")
best_n2_score = float('inf')
best_n2_params = None

# Search over angles and relative positions
for a1 in np.linspace(0, 180, 37):  # 5 degree increments
    for a2 in np.linspace(0, 180, 37):
        for x2 in np.linspace(-1, 1, 21):
            for y2 in np.linspace(-1, 1, 21):
                score = get_n2_score([x2, y2, a1, a2])
                if score < best_n2_score:
                    best_n2_score = score
                    best_n2_params = [x2, y2, a1, a2]

print(f"Best N=2 score from grid search: {best_n2_score:.6f}")
print(f"Best params: x2={best_n2_params[0]:.4f}, y2={best_n2_params[1]:.4f}, a1={best_n2_params[2]:.2f}, a2={best_n2_params[3]:.2f}")

# Current N=2 configuration
current_n2 = baseline_df[baseline_df['id'].str.startswith('002_')]
polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in current_n2.iterrows()]
current_n2_side = get_bounding_box_side(polygons)
current_n2_score = current_n2_side**2 / 2
print(f"\nCurrent N=2 score: {current_n2_score:.6f}")
print(f"Potential improvement: {current_n2_score - best_n2_score:.6f}")

In [ ]:
# Refine N=2 with local optimization
from scipy.optimize import minimize

def get_n2_score_for_opt(params):
    x2, y2, a1, a2 = params
    poly1 = create_tree_polygon(0, 0, a1)
    poly2 = create_tree_polygon(x2, y2, a2)
    
    if poly1.intersects(poly2) and not poly1.touches(poly2):
        intersection = poly1.intersection(poly2)
        if intersection.area > 1e-10:
            return 1000
    
    side = get_bounding_box_side([poly1, poly2])
    return side**2 / 2

# Refine best params
result = minimize(get_n2_score_for_opt, best_n2_params, method='Nelder-Mead', 
                  options={'maxiter': 10000, 'xatol': 1e-8, 'fatol': 1e-8})
print(f"Refined N=2 score: {result.fun:.6f}")
print(f"Refined params: {result.x}")

In [ ]:
# Use differential evolution for global optimization of N=2
from scipy.optimize import differential_evolution

bounds = [(-1.5, 1.5), (-1.5, 1.5), (0, 360), (0, 360)]

print("Running differential evolution for N=2...")
result_de = differential_evolution(get_n2_score_for_opt, bounds, maxiter=500, 
                                    seed=42, workers=-1, polish=True)
print(f"DE N=2 score: {result_de.fun:.6f}")
print(f"DE params: {result_de.x}")

# Verify no overlap
x2, y2, a1, a2 = result_de.x
poly1 = create_tree_polygon(0, 0, a1)
poly2 = create_tree_polygon(x2, y2, a2)
print(f"Overlap check: {poly1.intersects(poly2) and not poly1.touches(poly2)}")

In [ ]:
# Compare with current baseline for small N
print("\n" + "="*50)
print("COMPARISON WITH BASELINE")
print("="*50)

# N=1
print(f"\nN=1:")
print(f"  Current score: {current_side**2:.6f}")
print(f"  Best found: {best_side**2:.6f}")
print(f"  Improvement: {current_side**2 - best_side**2:.6f}")

# N=2
print(f"\nN=2:")
print(f"  Current score: {current_n2_score:.6f}")
print(f"  Best found: {result_de.fun:.6f}")
print(f"  Improvement: {current_n2_score - result_de.fun:.6f}")

total_improvement = (current_side**2 - best_side**2) + (current_n2_score - result_de.fun)
print(f"\nTotal potential improvement from N=1,2: {total_improvement:.6f}")

In [ ]:
# If we found improvements, create updated submission
improved_df = baseline_df.copy()

# Update N=1 if improved
if best_side**2 < current_side**2 - 1e-6:
    print(f"Updating N=1: angle {current_n1['deg_val']:.2f} -> {best_angle:.2f}")
    improved_df.loc[improved_df['id'] == '001_0', 'x'] = 's0.0'
    improved_df.loc[improved_df['id'] == '001_0', 'y'] = 's0.0'
    improved_df.loc[improved_df['id'] == '001_0', 'deg'] = f's{best_angle}'
    improved_df.loc[improved_df['id'] == '001_0', 'x_val'] = 0.0
    improved_df.loc[improved_df['id'] == '001_0', 'y_val'] = 0.0
    improved_df.loc[improved_df['id'] == '001_0', 'deg_val'] = best_angle

# Update N=2 if improved
if result_de.fun < current_n2_score - 1e-6:
    x2, y2, a1, a2 = result_de.x
    print(f"Updating N=2")
    # Tree 1 at origin
    improved_df.loc[improved_df['id'] == '002_0', 'x'] = 's0.0'
    improved_df.loc[improved_df['id'] == '002_0', 'y'] = 's0.0'
    improved_df.loc[improved_df['id'] == '002_0', 'deg'] = f's{a1}'
    improved_df.loc[improved_df['id'] == '002_0', 'x_val'] = 0.0
    improved_df.loc[improved_df['id'] == '002_0', 'y_val'] = 0.0
    improved_df.loc[improved_df['id'] == '002_0', 'deg_val'] = a1
    # Tree 2
    improved_df.loc[improved_df['id'] == '002_1', 'x'] = f's{x2}'
    improved_df.loc[improved_df['id'] == '002_1', 'y'] = f's{y2}'
    improved_df.loc[improved_df['id'] == '002_1', 'deg'] = f's{a2}'
    improved_df.loc[improved_df['id'] == '002_1', 'x_val'] = x2
    improved_df.loc[improved_df['id'] == '002_1', 'y_val'] = y2
    improved_df.loc[improved_df['id'] == '002_1', 'deg_val'] = a2

print("\nUpdated configurations")

In [ ]:
# Calculate new total score
new_total_score = 0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    group = improved_df[improved_df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    side = get_bounding_box_side(polygons)
    new_total_score += side**2 / n

print(f"New total score: {new_total_score:.6f}")
print(f"Baseline score: 70.676102")
print(f"Improvement: {70.676102 - new_total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {new_total_score - 68.919154:.6f}")

In [ ]:
# Save improved submission
improved_df[['id', 'x', 'y', 'deg']].to_csv('/home/submission/submission.csv', index=False)
print("Saved improved submission to /home/submission/submission.csv")

# Summary
print("\n" + "="*50)
print("EXPERIMENT 004 SUMMARY")
print("="*50)
print(f"Baseline score: 70.676102")
print(f"New score: {new_total_score:.6f}")
print(f"Improvement: {70.676102 - new_total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {new_total_score - 68.919154:.6f}")
print("="*50)